In [2]:
import os, json, collections, copy
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score

In [ ]:
filepath = '../data/buzzfeed_1267_gpt.csv'
df = pd.read_csv(filepath)
print(df.shape)
df.head(2)

filepath = '../data/buzzfeed182_gpt.csv'
df2 = pd.read_csv(filepath)
print(df2.shape)
df2.head(2)

In [ ]:
def get_mv_num(df):
    new_data = []
    for index, row in df.iterrows():
        dic = row.to_dict()
        claim_estimations = literal_eval(dic['claim_estimations'])
        support_or_negate_stat = {'support': 0, 'baseless': 0, 'negate': 0}
        for est in claim_estimations:
            support_or_negate = est['support_or_negate']
            support_or_negate_stat[f'{support_or_negate}'] += 1
        for k, v in support_or_negate_stat.items():
            dic[k] = v
        new_data.append(dic)
    print(len(new_data))
    new_df = pd.DataFrame(new_data)
    return new_df
new_df = get_mv_num(df)
new_df2 = get_mv_num(df2)

In [ ]:
new_df.head()

In [ ]:
keys = ['support', 'baseless', 'negate']
print('------------buzzfeed16k------------')
print(new_df.groupby('label')[keys].mean())
print('------------buzzfeed182------------')
print(new_df2.groupby('label')[keys].mean())

In [ ]:
total_data = []
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt.jsonl'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
print(len(total_data))

In [ ]:
total_data[0]['gpt'].keys()

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/exp/buzzfeed182_gpt/result.csv'
df = pd.read_csv(filepath)
print(df.shape)
df.head(1)

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt.csv'
df = pd.read_csv(filepath)
print(df.shape)
df.head(1)

## 没搜到的case

In [ ]:
total_data = []
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt.jsonl'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
print(len(total_data))

In [ ]:
no_fea = []
for d in total_data:
    overall_predictions = d['gpt']['features']['claim_estimations']
    if not overall_predictions:
        no_fea.append(d)

In [ ]:
np.max([])

In [ ]:
df = pd.read_csv('/home/services/siliang/ml_fakenews/data/buzzfeed_1267_gpt_v2.csv')
print(df.shape)

In [ ]:
total_data = []
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt.jsonl'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
# print(len(total_data))

In [ ]:
titletextset = set()
for d in total_data:
    titletextset.add(d['title']+d['text'])
print(len(titletextset))

In [ ]:
total_data = []
filepath = '../data/buzzfeed_1267_2model.jsonl'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
print(total_data)

titletextset = set()
for d in total_data:
    titletextset.add(d['title']+d['text'])
print(len(titletextset))

In [ ]:
print(len(titletextset))

## 特征重复问题

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt.csv'
df = pd.read_csv(filepath)

In [ ]:
df.shape

In [ ]:
## 导出case看下规律
filepath = '/home/services/siliang/ml_fakenews/exp/buzzfeed182_gpt_5fold/result.csv'
df = pd.read_csv(filepath)
print(df.shape)

In [ ]:
df.head()

In [ ]:
preds = list(df.preds.values)
labels = list(df.label.values)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
thresholds = [i/100 for i in range(101)]
best_f1 = 0
for thrd in thresholds:
    preds_bool = [i > thrd for i in preds]
    recall = recall_score(labels, preds_bool)
    precision = precision_score(labels, preds_bool)
    f1 = f1_score(labels, preds_bool)
    accuracy = accuracy_score(labels, preds_bool)
    if f1 > best_f1:
        best_thrd, best_recall, best_precision, best_f1, best_acc = thrd, recall, precision, f1, accuracy
print('thrd: {:.4f} recall: {:.4f}, precision: {:.4f}, f1: {:.4f}, accuracy: {:.4f}'.format(best_thrd, best_recall, best_precision, best_f1, best_acc))

In [ ]:
filepath = '../data/buzzfeed182_gpt.csv'
df = pd.read_csv(filepath)
df.head(1)

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_rerun_0314.csv'
df = pd.read_csv(filepath)
df.head(1)

In [ ]:
snset = set()
for index, row in df.iterrows():
    for info in literal_eval(row['claim_estimations']):
        support_or_negate = info['support_or_negate']
        snset.add(support_or_negate)
print(snset)

In [ ]:
def get_support_medianum(domain_list):

    support_list = []
    for d in domain_list:
        sn = d["support_or_negate"]
        sn = 1 if sn == "support" else -1 if sn == "negate" else 0
        media_quality = d["media_quality"]
        if not media_quality:
            media_quality = 3
        # confidence = d["confidence"]
        if sn == 1:
            support_list.append(media_quality)
    # print(support_list)
    if not support_list:
        return 0, 0, 0, 0, 0, 0, 0, 0
    return np.mean(support_list), np.max(support_list), np.min(support_list), np.sum(support_list), support_list.count(5), support_list.count(4), support_list.count(3), support_list.count(2), support_list.count(1)


df["claim_estimations"].apply(lambda x: pd.Series(get_support_medianum(literal_eval(x))))

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_rerun_0314_rationale.csv'
df = pd.read_csv(filepath)
df["gpt_rationale_score"] = df["gpt_rating"].apply(lambda x: eval(x)['rating'])# x['gpt_rating']['rating'])

In [ ]:
total_data = []
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_v2_rationale.jsonl'
savepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_v2_rationale.csv'

filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_rerun_0314_rationale.jsonl'
savepath= '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_rerun_0314_rationale.csv'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        if not isinstance(dic['gpt_rating'],dict):
            print(dic['gpt_rating'])
            if not isinstance(dic['gpt_rating']['rating'], int):
                print(dic['gpt_rating']['rating'])
        total_data.append(dic)
print(len(total_data))
df = pd.DataFrame(total_data)
df.to_csv(savepath)

In [ ]:
[i['rating'] for i in df.loc[:, 'gpt_rating']]

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/buzzfeed182_gpt_v2_rationale.csv'
df = pd.read_csv(filepath)
df.head(1)

In [ ]:
df["gpt_rationale_score"] = df["gpt_rating"].apply(lambda x: x['rating'])# x['gpt_rating']['rating'])

In [ ]:
df["gpt_rationale_score"] = df[["gpt_rating"]].apply(lambda x: literal_eval(x['gpt_rating'])['rating'], axis=1)

In [ ]:
filepath = '/home/services/siliang/ml_fakenews/data/politifact_547_gpt_main_claim_only_0315.csv'
df = pd.read_csv(filepath)
df.head(1)

In [ ]:
eval(df.loc[0, 'gpt'])['overall_predictions']

In [ ]:
eval(df.loc[0, 'gpt'])['features']

In [ ]:
df.label.value_counts()

In [ ]:
total_data = []
filepath = '/home/services/siliang/fakenews/FakeNewsNet-main/data_gen/politifact_547.jsonl'
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
print(len(total_data))

In [ ]:
df = pd.DataFrame(total_data)
df.label.value_counts()

In [ ]:
total_data[0]

In [5]:
filepath = '../data/politifact_547_gpt_main_claim_only_rationale_0315.jsonl'
total_data = []
with open(filepath) as f:
    for line in f:
        dic = json.loads(line)
        total_data.append(dic)
    print(len(total_data))

547


In [17]:
for data in total_data:
    gpt_score = data['gpt_rating']['rating']
    if isinstance(gpt_score, str):
        print(gpt_score)
        gpt_score = 5

N/A
